In [48]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)
import random
random.seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
from src.tradecli import *
from src.utils import *
from src.journal import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [50]:
display(full_df)

,type_1_m5,type_1_m1,type_1_m30,type_1_h4,type_1_m15,risk_reward_ratio,type_1_h1,type_1_d,type_1_M,type_1_w,has_type_1,has_type_2,has_type_3,has_time_frame
0,False,True,False,False,False,2.000000,False,False,False,False,True,False,False,False
1,False,False,False,False,False,0.666667,False,False,False,False,False,False,False,False
2,False,False,False,False,False,10.000000,False,False,False,False,False,False,False,False
3,False,False,False,False,False,10.000000,False,False,False,False,False,False,False,False
4,False,True,False,False,False,5.000000,False,False,False,False,True,False,False,False
5,False,False,False,False,False,5.913580,False,False,True,False,True,False,False,False
6,True,False,False,False,False,7.482759,False,False,False,False,True,False,False,False
7,False,False,False,False,True,2.215190,False,False,False,False,True,False,False,False
8,False,False,False,False,False,3.978022,False,False,True,False,True,False,False,False
9,False,False,False,False,True,1.348315,False,False,False,False,True,False,False,False


In [51]:
full_df.dtypes

type_1_m5               bool
type_1_m1               bool
type_1_m30              bool
type_1_h4               bool
type_1_m15              bool
risk_reward_ratio    float64
type_1_h1               bool
type_1_d                bool
type_1_M                bool
type_1_w                bool
has_type_1              bool
has_type_2              bool
has_type_3              bool
has_time_frame          bool
dtype: object

In [52]:
!rm -rf output

In [53]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
j.to_markdown(output_dir='output')

INFO:root:Converting trades to markdown
INFO:root:Files for trades found
INFO:root:Files Found #: 14, Total Trades #: 14max files for trade: 0min files for trade: 0avg files for trade: 0.0
INFO:root:Descriptive Statistics:           0
count  14.0
mean    0.0
std     0.0
min     0.0
25%     0.0
50%     0.0
75%     0.0
max     0.0
INFO:root:Writing index markdown
INFO:root:Calculating simple statistics


INFO:root:Analyzing tag relevance
INFO:root:Finding best single tags and subsets
INFO:root:Finding best single tags
INFO:root:Finding best tag subsets
INFO:root:Finding best tag subsets
INFO:root:Selecting top 10 features using RFE
INFO:root:Dropped 5 rows with NaN in 'outcome' y variable
INFO:root:X features # 42: {'type_3_m1', 'type_3_M', 'htf_poi_ltf_confirmation_False', 'type_2_m30', 'type_1_m5', 'type_1_m1', 'type_3_h4', 'type_3_w', 'htf_poi_ltf_confirmation', 'type_2_m15', 'type_3_m15', 'type_2_h4', 'close_price', 'type_1_m30', 'management_strategy', 'outcome_loss', 'type_3_d', 'type_2_M', 'type_2_m1', 'type_2_w', 'htf_poi_ltf_confirmation_True', 'type_3_h1', 'type_1_h4', 'type_2_m5', 'management_strategy_be_after_push', 'type_2_h1', 'management_strategy_close_early', 'type_1_m15', 'outcome_win', 'risk_reward_ratio', 'type_1_h1', 'management_strategy_be_after_1r', 'type_1_d', 'type_1_M', 'session', 'type_3_m5', 'type_3_m30', 'type_2_d', 'management_strategy_strategy_2', 'type_1_w